To dos  
1) Handle custom bin widths of histograms for categorical datasets  

In [1]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col

# ee.Authenticate()
ee.Initialize(project = "dri-apps")

Map = geemap.Map()

## Define Parameters and Load datasets

In [2]:
# ------------------------------------- Define parameters -----------------------------------------------

# Define time period to export
start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2021, 2, 15)


# -------------------------------- Define input Image Collection ----------------------------------------

# Define input dataset
# See dictionary below for list of input datasets
in_ic_name = 'Landsat'

# Define variable from dataset
# See dictionary below for variables available for each dataset
var_name = 'NDVI'


# ------------------------------- Define input Feature Collection ---------------------------------------

# Define input path for Feature Collection
in_fc_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'
in_fc = ee.FeatureCollection(in_fc_path)

# # Subset by geometry
# geometry = ee.Geometry.Polygon([[[-108.4020, 38.7855], [-108.4020, 39.6080], [-109.1823, 39.6080], [-109.1823, 38.7855]]], None, False);
# in_fc = in_fc.filterBounds(geometry)


# ------------------------------ Define mask, if applicable --------------------------------------------

# Apply mask for ownership, landcover, or other variables. Must be binary mask.
mask = True
mask_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-sma-binary'


# ---------------------------- Additional parameters derived from above --------------------------------

# Define input Image Collection variables using dataset dictionary
in_ic_dict = {'GridMET_Drought': {'in_ic_paths': ['GRIDMET/DROUGHT'],
                                  'var_names': ['Long_Term_Drought_Blend', 'Short_Term_Drought_Blend'],
                                  'var_type': 'Categorical'},
              'GridMET': {'in_ic_paths': ['IDAHO_EPSCOR/GRIDMET'],
                          'var_names': ['precip', 'tmmn', 'tmmx', 'eto', 'vpd', 'windspeed', 'srad'],
                          'var_type': 'Continuous'},
              'RAP_Cover': {'in_ic_paths': ['projects/rap-data-365417/assets/vegetation-cover-v3'],
                            'var_names': ['AFG', 'BGR', 'LTR', 'PFG', 'SHR', 'TRE'],
                            'var_type': 'Continuous'},
              'RAP_Production': {'in_ic_paths': ['projects/rap-data-365417/assets/npp-partitioned-v3'],
                                 'var_names': ['afgAGB', 'pfgAGB', 'herbaceousAGB'],
                                 'var_type': 'Continuous'},
              'RAP_16dProduction': {'in_ic_paths': ['projects/rap-data-365417/assets/npp-partitioned-16day-v3'],
                                   'var_names': ['afgAGB', 'pfgAGB', 'herbaceousAGB'],
                                   'var_type': 'Continuous'},
              'USDM': {'in_ic_paths': ['projects/climate-engine/usdm/weekly'],
                       'var_names': ['drought'],
                       'var_type': 'Categorical'},
              'MOD11_LST': {'in_ic_paths': ['MODIS/061/MOD11A2'],
                            'var_names': ['LST_Day_1km'],
                            'var_type': 'Continuous'},
              'Landsat': {'in_ic_paths': ['LANDSAT/LT05/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LC09/C02/T1_L2'],
                          'var_names': ['NDVI'],
                          'var_type': 'Continuous'},
              'MOD16_ET': {'in_ic_paths': ['MODIS/006/MOD16A2'],
                           'var_names': ['ET', 'PET'],
                           'var_type': 'Continuous'},
              'MTBS': {'in_ic_paths': ['projects/climate-engine-pro/assets/mtbs_mosaics_annual'],
                       'var_names': ['Severity'],
                       'var_type': 'Categorical'}}

# Define properties for variables in dictionary
var_dict = {'Long_Term_Drought_Blend': {'units': 'drought'},
            'Short_Term_Drought_Blend': {'units': 'drought'},
            'precip': {'units': 'mm'},
            'tmmn': {'units': 'degrees C'},
            'tmmx': {'units': 'degrees C'},
            'eto': {'units': 'mm'},
            'vpd': {'units': 'kPa'},
            'windspeed': {'units': 'm/s'},
            'srad': {'units': 'W/m^2'},
            'AFG': {'units': '% cover'},
            'BGR': {'units': '% cover'},
            'LTR': {'units': '% cover'},
            'PFG': {'units': '% cover'},
            'SHR': {'units': '% cover'},
            'TRE': {'units': '% cover'},
            'afgAGB': {'units': 'lbs/acre'},
            'pfgAGB': {'units': 'lbs/acre'},
            'herbaceousAGB': {'units': 'lbs/acre'},
            'drought': {'units': 'drought'},
            'LST_Day_1km': {'units': 'degrees C'},
            'NDVI': {'units': 'unitless'},
            'ET': {'units': 'mm'},
            'PET': {'units': 'mm'},
            'Severity': {'units': 'fire severity'}}

# Define land unit names
if(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-grazing-allotment-polygons'):
    land_unit_long = 'BLM_Natl_Grazing_Allotment_Polygons'
    land_unit_short = 'BLM_Allotments'
    in_fc_id = 'ALLOT_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-stateoffice-polygons'):
    land_unit_long = 'BLM_Natl_FieldOffice_Polygons'
    land_unit_short = 'BLM_FieldOffices'
    in_fc_id = 'FO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'):
    land_unit_long = 'BLM_Natl_DistrictOffice_Polygons'
    land_unit_short = 'BLM_DistrictOffices'
    in_fc_id = 'DO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-fieldoffice-polygons'):
    land_unit_long = 'BLM_Natl_StateOffice_Polygons'
    land_unit_short = 'BLM_StateOffices'
    in_fc_id = 'SO_ID'

# Pull out additional variables needed to run exports
in_ic_paths = in_ic_dict.get(in_ic_name).get('in_ic_paths')
in_ic_res = ee.Number(ee.ImageCollection(in_ic_paths[0]).first().projection().nominalScale()).round().getInfo()
var_type = in_ic_dict.get(in_ic_name).get('var_type')
var_units = var_dict.get(var_name).get('units')
out_path = f"projects/dri-apps/assets/blm-database/{land_unit_short.replace('_', '').lower()}-{in_ic_name.replace('_', '').lower()}-{var_name.replace('_', '').lower()}"

## Create database image collection and append time-series images

In [3]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units, 'mask': mask}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    
    # Apply ID image function to input feature collection
    out_list = eedb_cor.generate_id_img(in_fc = in_fc, in_fc_id = in_fc_id)
    out_i = ee.Image(out_list.get(0))
    out_fc = ee.FeatureCollection(out_list.get(1))
    
    # Generate empty Image Collection asset to append images
    os.system(f"earthengine create collection {out_path}")
    
    # Export ID image to new Image Collection
    task = ee.batch.Export.image.toAsset(
        image = out_i.set(properties),
        description = f"initialize - {land_unit_short.replace('_', '').lower()} {in_ic_name.replace('_', '').lower()} {var_name.replace('_', '').lower()} - id",
        assetId = out_path + '/0_id',
        region = out_fc.geometry().buffer(20),
        scale = 22.264,
        maxPixels = 1e13)
    task.start()

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:

    print(f"Appending to Image Collection for dates {start_date} - {end_date}")

    
    # ----- Preprocess input Image Collection based on path for each date -----

    dates = eedb_col.get_collection_dates(in_ic_paths, start_date, end_date)

    for date in dates:
        print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
        
        date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')
        

        if in_ic_paths == ['GRIDMET/DROUGHT']:

            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_gm_drought(in_ic_paths, var_name, date)
  
        elif in_ic_paths == ['IDAHO_EPSCOR/GRIDMET']:

            # Run function to pre-process the GridMET data
            in_i = eedb_col.preprocess_gm(in_ic_paths, var_name, date)

        elif in_ic_paths == ['projects/rap-data-365417/assets/vegetation-cover-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-16day-v3']:

            # Run function to pre-process the RAP data
            in_i = eedb_col.preprocess_rap(in_ic_paths, var_name, date)

        elif in_ic_paths == ['projects/climate-engine/usdm/weekly']:

            # Run function to pre-process the USDM data
            in_i = eedb_col.preprocess_usdm(in_ic_paths, var_name, date)

        elif in_ic_paths == ['MODIS/061/MOD11A2']:

            # Run function to pre-process the MODIS LST data
            in_i = eedb_col.preprocess_modlst(in_ic_paths, var_name, date)

        elif in_ic_paths == ['LANDSAT/LT05/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LC09/C02/T1_L2']:

            # Run function to pre-process the Landsat SR NDVI data
            in_i = eedb_col.preprocess_lsndvi(in_ic_paths, var_name, date, in_fc)

        elif in_ic_paths == ['MODIS/006/MOD16A2']:

            # Run function to pre-process the MODIS ET data
            in_i = eedb_col.preprocess_modet(in_ic_paths, var_name, date)

        elif in_ic_paths == ['projects/climate-engine-pro/assets/mtbs_mosaics_annual']:

            # Run function to pre-process the MTBS data
            in_i = eedb_col.preprocess_mtbs(in_ic_paths, var_name, date)


        # ---------------------------- Apply functions to output image ---------------------------------

        # Conditionally apply mask to images
        if mask == True:

            # Select date band for single date
            in_i = in_i.updateMask(ee.Image(mask_path))

        elif mask == False:

            # Select date band for single date
            in_i = in_i

        # Create dictionary of properties        
        properties = {'system:index': date_ymd, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                      'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                      'var_name': var_name, 'var_units': var_units, 'mask': mask}
        
        # Conditionally add mask path to properties
        if mask == True:
            properties['mask_path'] = mask_path
        elif mask == False:
            properties['mask_path'] = 'None'

        if var_type == 'Continuous':

            # Run function to get time-series statistics for input feature collection
            out_fc = eedb_cor.img_to_pts_continuous(in_i, in_fc)

            # Convert centroid time-series to image collection time-series
            out_i = eedb_cor.pts_to_img_continuous(in_fc = out_fc)

        elif var_type == 'Categorical':

            # Run function to get time-series statistics for input feature collection for continuous variables
            out_fc = eedb_cor.img_to_pts_categorical(in_i, in_fc, in_ic_name = in_ic_name)

            # Convert centroid time-series to image collection time-series
            out_i = eedb_cor.pts_to_img_categorical(in_fc = out_fc, in_ic_name = in_ic_name)

        # Create out region for export
        out_region = out_fc.geometry().buffer(20)

        # Export the image
        eedb_cor.export_img(out_i = out_i, out_region = out_region, out_path = out_path, out_id = date_ymd, properties = properties)

{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-landsat-ndvi",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-landsat-ndvi",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-07T18:28:44.331016Z"
}
{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-landsat-ndvi",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-landsat-ndvi",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-07T18:28:44.331016Z"
}
Appending to Image Collection for dates 2021-01-01 00:00:00 - 2021-02-15 00:00:00
Running  2020-12-31 17:00:00
Running  2021-01-16 17:00:00
Running  2021-02-01 17:00:00


### Attempt to loop over years to produce exports to reduce pre-processing burden

In [4]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units, 'mask': mask}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    
    # Apply ID image function to input feature collection
    out_list = eedb_cor.generate_id_img(in_fc = in_fc, in_fc_id = in_fc_id)
    out_i = ee.Image(out_list.get(0))
    out_fc = ee.FeatureCollection(out_list.get(1))
    
    # Generate empty Image Collection asset to append images
    os.system(f"earthengine create collection {out_path}")
    
    # Export ID image to new Image Collection
    task = ee.batch.Export.image.toAsset(
        image = out_i.set(properties),
        description = f"initialize - {land_unit_short.replace('_', '').lower()} {in_ic_name.replace('_', '').lower()} {var_name.replace('_', '').lower()} - id",
        assetId = out_path + '/0_id',
        region = out_fc.geometry().buffer(20),
        scale = 22.264,
        maxPixels = 1e13)
    task.start()

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:
    
    for year in range(start_date.year, end_date.year):
        
        start_date_inner = datetime.datetime(year, 1, 1)
        end_date_inner = datetime.datetime(year+1, 1, 1)
        
        print(f"Appending to Image Collection for dates {start_date_inner} - {end_date_inner}")
        
        # ----- Preprocess input Image Collection based on path -----
        
        if in_ic_paths == ['GRIDMET/DROUGHT']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_gm_drought(in_ic_paths, var_name, start_date_inner, end_date_inner)
            
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
            
        elif in_ic_paths == ['IDAHO_EPSCOR/GRIDMET']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_gm(in_ic_paths, var_name, start_date_inner, end_date_inner)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
        
        elif in_ic_paths == ['projects/rap-data-365417/assets/vegetation-cover-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-16day-v3']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_rap(in_ic_paths, var_name, start_date_inner, end_date_inner)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
    
        elif in_ic_paths == ['projects/climate-engine/usdm/weekly']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_usdm(in_ic_paths, var_name, start_date_inner, end_date_inner)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
    
        elif in_ic_paths == ['MODIS/061/MOD11A2']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_modlst(in_ic_paths, var_name, start_date_inner, end_date_inner)
            
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
        
        elif in_ic_paths == ['LANDSAT/LT05/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LC09/C02/T1_L2']:
        
            # Run function to pre-process the Landsat SR NDVI data
            in_i = eedb_col.preprocess_lsndvi(in_ic_paths, var_name, start_date_inner, end_date_inner, in_fc)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
            
        elif in_ic_paths == ['MODIS/006/MOD16A2']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_modet(in_ic_paths, var_name, start_date_inner, end_date_inner)
            
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
        
        elif in_ic_paths == ['projects/climate-engine-pro/assets/mtbs_mosaics_annual']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_mtbs(in_ic_paths, var_name, start_date, end_date)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
    
    
        # ---------------------------- Iterate over in_dates with functions ---------------------------------
            
        for in_date in in_dates:
        
            print('Running ' + in_date)
            
            # Conditionally apply mask to images
            if mask == True:
                
                # Select date band for single date
                in_i_date = in_i.select([in_date]).updateMask(ee.Image(mask_path))
                
            elif mask == False:
                
                # Select date band for single date
                in_i_date = in_i.select([in_date])
        
            # Create dictionary of properties        
            properties = {'system:index': in_date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                          'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                          'var_name': var_name, 'var_units': var_units, 'mask': mask}
            if mask == True:
                properties['mask_path'] = mask_path
            elif mask == False:
                properties['mask_path'] = 'None'
    
            if var_type == 'Continuous':
            
                # Run function to get time-series statistics for input feature collection
                out_fc = eedb_cor.img_to_pts_continuous(in_i_date, in_fc)
    
                # Convert centroid time-series to image collection time-series
                out_i = eedb_cor.pts_to_img_continuous(in_fc = out_fc)
            
            elif var_type == 'Categorical':
            
                # Run function to get time-series statistics for input feature collection for continuous variables
                out_fc = eedb_cor.img_to_pts_categorical(in_i_date, in_fc, in_ic_name = in_ic_name)
           
                # Convert centroid time-series to image collection time-series
                out_i = eedb_cor.pts_to_img_categorical(in_fc = out_fc, in_ic_name = in_ic_name)
            
            # Create out region for export
            out_region = out_fc.geometry().buffer(20)
            
            # Export the image
            eedb_cor.export_img(out_i = out_i, out_region = out_region, out_path = out_path, out_id = in_date, properties = properties)

{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-tmmn",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-tmmn",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-07T04:00:33.755324Z"
}
{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-tmmn",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-tmmn",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-07T04:00:33.755324Z"
}
Appending to Image Collection for dates 1990-01-01 00:00:00 - 1991-01-01 00:00:00
Running 19900105
Running 19900110
Running 19900115
Running 19900120
Running 19900125
Running 19900130
Running 19900204
Running 19900209
Running 19900214
Running 19900219
Running 19900224
Running 19900301
Running 19900306
Running 19900311
Running 19900316
Running 19900321
Running 19900326
Running 19900331
Running 19900405
Running 19900410
Running 19900415
Running 19900420
Running 19900425
Running 19900430
Running 19900505
Runni

Running 19951106
Running 19951111
Running 19951116
Running 19951121
Running 19951126
Running 19951201
Running 19951206
Running 19951211
Running 19951216
Running 19951221
Running 19951226
Running 19951231
Appending to Image Collection for dates 1996-01-01 00:00:00 - 1997-01-01 00:00:00
Running 19960105
Running 19960110
Running 19960115
Running 19960120
Running 19960125
Running 19960130
Running 19960204
Running 19960209
Running 19960214
Running 19960219
Running 19960224
Running 19960229
Running 19960305
Running 19960310
Running 19960315
Running 19960320
Running 19960325
Running 19960330
Running 19960404
Running 19960409
Running 19960414
Running 19960419
Running 19960424
Running 19960429
Running 19960504
Running 19960509
Running 19960514
Running 19960519
Running 19960524
Running 19960529
Running 19960603
Running 19960608
Running 19960613
Running 19960618
Running 19960623
Running 19960628
Running 19960703
Running 19960708
Running 19960713
Running 19960718
Running 19960723
Running 19960728


Running 20020105
Running 20020110
Running 20020115
Running 20020120
Running 20020125
Running 20020130
Running 20020204
Running 20020209
Running 20020214
Running 20020219
Running 20020224
Running 20020301
Running 20020306
Running 20020311
Running 20020316
Running 20020321
Running 20020326
Running 20020331
Running 20020405
Running 20020410
Running 20020415
Running 20020420
Running 20020425
Running 20020430
Running 20020505
Running 20020510
Running 20020515
Running 20020520
Running 20020525
Running 20020530
Running 20020604
Running 20020609
Running 20020614
Running 20020619
Running 20020624
Running 20020629
Running 20020704
Running 20020709
Running 20020714
Running 20020719
Running 20020724
Running 20020729
Running 20020803
Running 20020808
Running 20020813
Running 20020818
Running 20020823
Running 20020828
Running 20020902
Running 20020907
Running 20020912
Running 20020917
Running 20020922
Running 20020927
Running 20021002
Running 20021007
Running 20021012
Running 20021017
Running 200210

Running 20080320
Running 20080325
Running 20080330
Running 20080404
Running 20080409
Running 20080414
Running 20080419
Running 20080424
Running 20080429
Running 20080504
Running 20080509
Running 20080514
Running 20080519
Running 20080524
Running 20080529
Running 20080603
Running 20080608
Running 20080613
Running 20080618
Running 20080623
Running 20080628
Running 20080703
Running 20080708
Running 20080713
Running 20080718
Running 20080723
Running 20080728
Running 20080802
Running 20080807
Running 20080812
Running 20080817
Running 20080822
Running 20080827
Running 20080901
Running 20080906
Running 20080911
Running 20080916
Running 20080921
Running 20080926
Running 20081001
Running 20081006
Running 20081011
Running 20081016
Running 20081021
Running 20081026
Running 20081031
Running 20081105
Running 20081110
Running 20081115
Running 20081120
Running 20081125
Running 20081130
Running 20081205
Running 20081210
Running 20081215
Running 20081220
Running 20081225
Running 20081230
Appending to I

Running 20140604
Running 20140609
Running 20140614
Running 20140619
Running 20140624
Running 20140629
Running 20140704
Running 20140709
Running 20140714
Running 20140719
Running 20140724
Running 20140729
Running 20140803
Running 20140808
Running 20140813
Running 20140818
Running 20140823
Running 20140828
Running 20140902
Running 20140907
Running 20140912
Running 20140917
Running 20140922
Running 20140927
Running 20141002
Running 20141007
Running 20141012
Running 20141017
Running 20141022
Running 20141027
Running 20141101
Running 20141106
Running 20141111
Running 20141116
Running 20141121
Running 20141126
Running 20141201
Running 20141206
Running 20141211
Running 20141216
Running 20141221
Running 20141226
Running 20141231
Appending to Image Collection for dates 2015-01-01 00:00:00 - 2016-01-01 00:00:00
Running 20150105
Running 20150110
Running 20150115
Running 20150120
Running 20150125
Running 20150130
Running 20150204
Running 20150209
Running 20150214
Running 20150219
Running 20150224


Running 20200822
Running 20200827
Running 20200901
Running 20200906
Running 20200911
Running 20200916
Running 20200921
Running 20200926
Running 20201001
Running 20201006
Running 20201011
Running 20201016
Running 20201021
Running 20201026
Running 20201031
Running 20201105
Running 20201110
Running 20201115
Running 20201120
Running 20201125
Running 20201130
Running 20201205
Running 20201210
Running 20201215
Running 20201220
Running 20201225
Running 20201230
Appending to Image Collection for dates 2021-01-01 00:00:00 - 2022-01-01 00:00:00
Running 20210105
Running 20210110
Running 20210115
Running 20210120
Running 20210125
Running 20210130
Running 20210204
Running 20210209
Running 20210214
Running 20210219
Running 20210224
Running 20210301
Running 20210306
Running 20210311
Running 20210316
Running 20210321
Running 20210326
Running 20210331
Running 20210405
Running 20210410
Running 20210415
Running 20210420
Running 20210425
Running 20210430
Running 20210505
Running 20210510
Running 20210515
